# Running ASFMAUS

In [ ]:
using StochasticDiffEq
using DiffEqCallbacks
using DifferentialEquations
using Plots
using LinearAlgebra
using DelimitedFiles
using DataFrames
using LinearAlgebra
using SparseArrays
using Distributions

In [ ]:
include("/home/callum/ASF/Modules/ASF_input.jl");
include("/home/callum/ASF/Modules/ASF_model.jl");
include("/home/callum/ASF/Modules/ASF_analysis.jl");

In [ ]:
input_path = "/home/callum/ASF/Inputs/";

In [ ]:
input_1 = ASF_Inputs.Model_Data(input_path); #creating parameters to run model

In [ ]:
nt = input_1.Parameters.Populations.cum_sum[end] #total number of groups and farms
nc = 5 #number of classes
eqs = 11 #number of processes
n_ens = 100 #number of runs in ensemble

dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+1:nc*nt*eqs+nc:end] .= 1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+4:nc*nt*eqs+nc:end] .= -1


dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [ ]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
    nothing
end

function prob_func(prob,i,repeat) #function to change parameters while keeping network the same
    ASF_Model.reparam!(input_1)
    remake(prob,p=input_1.Parameters)
end 

## Running the Model

### Frequency run

In [ ]:
include("/home/callum/ASF/Modules/ASF_input.jl");
include("/home/callum/ASF/Modules/ASF_model.jl");

In [ ]:
input_1 = ASF_Inputs.Model_Data(input_path); #creating parameters to run model

In [ ]:
u0 = input_1.U0
u0[1:5:end] += u0[2:5:end] +u0[3:5:end]
u0[2:5:end] .= 0
u0[3:5:end] .= 0;



In [ ]:
N_years = 3
n_ens = 10

rj = RegularJump(ASF_Model.density_rate, regular_c, eqs*nt)

prob = DiscreteProblem(input_1.U0,(0.0,365.0*N_years),input_1.Parameters)

jump_prob = JumpProblem(prob,Direct(),rj)

sol = solve(jump_prob, SimpleTauLeaping(), dt= 1)#callback=DiscreteCallback(pcondit,paffect!), dt=1 ) ;
#ensemble_prob_m = EnsembleProblem(jump_prob)#,prob_func=prob_func)
#output = solve(ensemble_prob_m, SimpleTauLeaping(), dt=1, trajectories= n_ens);

In [ ]:
n_sims = 100
final_pops = Vector{Int16}(undef, n_sims)


for i in 1:n_sims
    sol = output[i]
    data = reduce(vcat,transpose.(sol.u))
    
    if any(x->x <0, data)
        println("Need to Reduce Timestep")
        data[data .< 0 ] .= 0
    end
    
    s_d = data[:,1:5:end]
    e_d = data[:,2:5:end]
    i_d = data[:,3:5:end]
    r_d = data[:,4:5:end]
    c_d = data[:,5:5:end]
    
    disease = e_d + i_d + c_d #classes with disease
    disease_free = s_d + r_d #classes without disease
    
    disease_sum = sum(disease,dims=2)
    pop_sum = sum(disease+disease_free,dims=2)
    
    N = s_d + e_d + i_d+ r_d + c_d;

    final_pop = sum(N[end,:])
    final_pops[i] = final_pop
end 

## Running Multiple Networks

In [ ]:
function sensitivity(n_networks, n_dist, n_runs, path, name)
    
    input_folder = splitpath(path)[end]
    base_path = rsplit(path, input_folder)[1]

    #making save directory
    dir = "$(base_path)Results/$(name)"

    isdir(dir) || mkdir(dir)
    cd(dir)

    #copying inputs
    cp("$(path)", "Inputs",force = true)
    
    for i = 1:n_networks
        inputs = ASF_Inputs.Model_Data(path); #creating parameters to run model
        for j = 1:n_dist

            rj = RegularJump(ASF_Model.density_rate,regular_c,eqs*nt)
            jumps = JumpSet(rj)
            prob = DiscreteProblem(inputs.U0,inputs.Time,inputs.Parameters)
            jump_prob = JumpProblem(prob,Direct(),rj) 
            ensemble_prob_m = EnsembleProblem(jump_prob)
            outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

            summary = ASF_Analysis.quick_analysis(outputs, inputs) #update to new function from module (ASF_Model.quick_analysis(outputs))

            writedlm( "Results_$(i)_$(j).csv",  summary, ',')
            ASF_Model.reparam!(inputs) #re-doing params with
        end
        println("$(100*i/n_networks) %")

    end
    
end

In [ ]:
sensitivity(20,20,100,input_path, "Dataout") # running 

## Analysing Output

In [ ]:
density = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Baseline//");
connect_5 = ASF_Analysis.read_outputs("/home/callum/ASF/Results/Connect_5//");


In [ ]:
density_r = ASF_Analysis.three_statistics(density,20, p_out = true);

In [ ]:
connect_5_r = ASF_Analysis.three_statistics(connect_5,20, p_out = true);